In [1]:
import torch
import fastai

class ImagenetDataset(torch.utils.data.Dataset):

    def __init__(self, path, df, label_dict, dim = 256, transforms=None):
        self.df = df
        self.transforms = transforms
        self.image_paths = get_image_files(path)
        self.label_dict = label_dict
        self.dim = dim

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        label = self.label_dict[self.df[self.df["image"] == img_path.name]["label"].values[0]]
        img = A.Resize(self.dim, self.dim)(image=img)["image"]

        if self.transforms is not None:
            img = self.transforms(image=img)["image"]

        img = F.to_tensor(img)
        label = torch.as_tensor(label)
        label = torch.nn.functional.one_hot(label,len(self.df["label"].unique()))

        return img, label.float()